In [1]:
import ee

ee.Authenticate()

#每一次打开都要重新激活

True

# Assets管理

In [4]:
import ee
ee.Initialize()

SRC_ROOT = 'projects/ee-yaoyansum/assets'
DST_ROOT = f'{SRC_ROOT}/Rubbish'

# 列出源目录下所有资产
assets = ee.data.listAssets({'parent': SRC_ROOT})['assets']

for asset in assets:
    asset_id = asset['id']
    # 如果是目标文件夹自身，就跳过
    if asset_id == DST_ROOT:
        continue
    name = asset_id.split('/')[-1]
    new_id = f"{DST_ROOT}/{name}"
    print(f"Moving {asset_id} → {new_id}")
    ee.data.renameAsset(asset_id, new_id)


Moving projects/ee-yaoyansum/assets/04_0_11053 → projects/ee-yaoyansum/assets/Rubbish/04_0_11053
Moving projects/ee-yaoyansum/assets/05_0_15649 → projects/ee-yaoyansum/assets/Rubbish/05_0_15649
Moving projects/ee-yaoyansum/assets/06_1_10096 → projects/ee-yaoyansum/assets/Rubbish/06_1_10096
Moving projects/ee-yaoyansum/assets/06_2_10096 → projects/ee-yaoyansum/assets/Rubbish/06_2_10096
Moving projects/ee-yaoyansum/assets/07_1_10157 → projects/ee-yaoyansum/assets/Rubbish/07_1_10157
Moving projects/ee-yaoyansum/assets/07_2_10158 → projects/ee-yaoyansum/assets/Rubbish/07_2_10158
Moving projects/ee-yaoyansum/assets/08_1_10212 → projects/ee-yaoyansum/assets/Rubbish/08_1_10212
Moving projects/ee-yaoyansum/assets/08_2_10213 → projects/ee-yaoyansum/assets/Rubbish/08_2_10213
Moving projects/ee-yaoyansum/assets/09_1_10138 → projects/ee-yaoyansum/assets/Rubbish/09_1_10138
Moving projects/ee-yaoyansum/assets/09_2_10138 → projects/ee-yaoyansum/assets/Rubbish/09_2_10138
Moving projects/ee-yaoyansum/a

EEException: Before trying to move an asset, delete its children: 'projects/ee-yaoyansum/assets/20250512Advance'.

In [3]:
#上传时一定要有ID列


import ee
import math
ee.Initialize()


# ==================== 数据预处理 ====================
def preprocess_landsat(collection, band_map):
    """统一Landsat各卫星波段名称（区分不同卫星）"""
    return collection.select(band_map['input'], band_map['output'])

# 定义各卫星的波段映射关系
# Landsat 5/7的波段映射（原始波段 → 统一名称）
landsat5_7_bands = {
    'input': ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL'],
    'output': ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL']
}

# Landsat 8/9的波段映射（保持原始名称，无需重命名）
landsat8_9_bands = {
    'input': ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL'],
    'output': ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL']
}

# 预处理各卫星数据（应用不同的波段映射）
landsat5 = preprocess_landsat(ee.ImageCollection("LANDSAT/LT05/C02/T1_L2"), landsat5_7_bands)
landsat7 = preprocess_landsat(ee.ImageCollection("LANDSAT/LE07/C02/T1_L2"), landsat5_7_bands)
landsat8 = preprocess_landsat(ee.ImageCollection("LANDSAT/LC08/C02/T1_L2"), landsat8_9_bands)
landsat9 = preprocess_landsat(ee.ImageCollection("LANDSAT/LC09/C02/T1_L2"), landsat8_9_bands)

# 合并多卫星数据
merged = landsat5.merge(landsat7).merge(landsat8).merge(landsat9)

# ==================== 特征集合加载 ====================
table = ee.FeatureCollection("projects/ee-yaoyansum/assets/ZYY_co2/zyy")

# ==================== 核心处理函数 ====================
def apply_qa_mask(image):
    """基于位运算的水体掩膜优化"""
    qa = image.select('QA_PIXEL')
    
    # ===== 核心掩膜条件 =====
    # 水体条件：检测第7位（位7对应水体质心标记）
    water_mask = qa.bitwiseAnd(1 << 7)  # 位7为水体质心标记
    
    # ===== 干扰排除条件 =====
    # 排除填充像元（位1）
    fill_mask = qa.bitwiseAnd(1 << 1).eq(0)
    # 排除云阴影（位2-3：00表示无阴影）
    shadow_mask = qa.bitwiseAnd(0b1100).eq(0)
    # 排除云（位4-5：00表示无云）
    cloud_mask = qa.bitwiseAnd(0b110000).eq(0)
    # 排除雪/冰（位5）
    snow_mask = qa.bitwiseAnd(1 << 5).eq(0)
    
    # ===== 组合掩膜 =====
    final_mask = water_mask.And(fill_mask) \
                          .And(shadow_mask) \
                          .And(cloud_mask) \
                          .And(snow_mask)
    
    return image.updateMask(final_mask).copyProperties(image, ['system:time_start', 'SPACECRAFT_ID'])


def convert_float_date(feature):
    """将浮点型YYYYMMDD转换为日期对象"""
    date_float = ee.Number(feature.get('date'))
    date_str = date_float.toInt().format('%08d')  # 转换为整数再格式化
    return feature.set({
        'parsed_date': ee.Date.parse('YYYYMMdd', date_str, 'Asia/Shanghai')
    })

def process_batch(batch_index, total_batches):
    """处理单个批次"""
    start = batch_index * batch_size
    end = ee.Number(start).add(batch_size).min(total_features)
    current_batch = feature_list.slice(start, end)
    
    # 日期转换
    processed_collection = ee.FeatureCollection(current_batch).map(convert_float_date)
    
    def match_images(feature):
        """匹配影像与采样点"""
        feature_date = ee.Date(feature.get('parsed_date'))
        point = ee.Geometry.Point([feature.getNumber('Lon'), feature.getNumber('Lat')])
        
        # 时间窗口±7天
        date_range = ee.DateRange(
            feature_date.advance(-7, 'day'),
            feature_date.advance(7, 'day')
        )
        
        # 筛选影像
        filtered = merged \
            .filterBounds(point) \
            .filterDate(date_range) \
            .map(apply_qa_mask)
        
        def extract_data(image):
            """提取波段值"""
            stats = image.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=point,
                scale=30,
                maxPixels=1e9
            )
            
            return ee.Feature(None, {
                # 'HYBAS_ID': feature.get('HYBAS_ID'),
                'Field_date': feature.get('date'),
                'Hy_id': feature.get('Hy_id'),
                'Lon': feature.get('Lon'),
                'Lat': feature.get('Lat'),
                'Dataset': feature.get('dataset'),
                'Hylak_id': feature.get('Hylak_id'),
                'Satellite': image.get('SPACECRAFT_ID'),
                'Acq_Date': ee.Date(image.get('system:time_start')).format('YYYY-MM-dd'),
                'B2': stats.get('SR_B2'),
                'B3': stats.get('SR_B3'),
                'B4': stats.get('SR_B4'),
                'B5': stats.get('SR_B5'),
                'B6': stats.get('SR_B6'),
                'B7': stats.get('SR_B7'),
                'QA_Valid': stats.contains('SR_B2')  # 有效性标志
            })
        
        # 过滤无效结果
        return filtered.map(extract_data).filter(ee.Filter.eq('QA_Valid', True))
    
    # 执行并导出
    results = processed_collection.map(match_images).flatten()
    
    # 导出配置（根据实际字段名调整）
    task = ee.batch.Export.table.toDrive(
        collection=ee.FeatureCollection(results),
        description=f'WeatherData_Hourly_ERA5_Land_ZYY_RRS5',
        folder='American_CO2',
        fileFormat='CSV',
        selectors=[
             'Field_date','Hy_id', 'Lon', 'Lat','Dataset','Hylak_id',
            'Satellite', 'Acq_Date', 'B2', 'B3', 
            'B4', 'B5', 'B6', 'B7'
        ]
    )
    task.start()

# ==================== 执行分批次处理 ====================
if __name__ == "__main__":
    feature_list = table.toList(table.size())
    total_features = feature_list.size().getInfo()
    total_batches = 1  # 根据数据量调整
    
    batch_size = math.ceil(total_features / total_batches)
    
    for i in range(total_batches):
        process_batch(i, total_batches)
        print(f'已提交批次 {i+1}/{total_batches}')

已提交批次 1/1


# 润色代码

In [13]:
import ee
import math

ee.Initialize()

# ==================== 数据预处理 ====================
def preprocess_landsat(collection, band_map):
    return collection.select(band_map['input'], band_map['output'])

# 波段映射
landsat5_7_bands = {
    'input': ['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B7','QA_PIXEL'],
    'output': ['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7','QA_PIXEL']
}
landsat8_9_bands = {
    'input': ['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7','QA_PIXEL'],
    'output': ['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7','QA_PIXEL']
}

# 预处理集合
landsat5 = preprocess_landsat(ee.ImageCollection("LANDSAT/LT05/C02/T1_L2"), landsat5_7_bands)
landsat7 = preprocess_landsat(ee.ImageCollection("LANDSAT/LE07/C02/T1_L2"), landsat5_7_bands)
landsat8 = preprocess_landsat(ee.ImageCollection("LANDSAT/LC08/C02/T1_L2"), landsat8_9_bands)
landsat9 = preprocess_landsat(ee.ImageCollection("LANDSAT/LC09/C02/T1_L2"), landsat8_9_bands)

# ==================== 反射率转换 ====================
def scale_to_reflectance(image):
    optical = image.select(['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7'])
    scaled = optical.multiply(0.0000275).add(-0.2)
    return image.addBands(scaled.rename(['Rrs_B2','Rrs_B3','Rrs_B4','Rrs_B5','Rrs_B6','Rrs_B7']))

# ==================== 添加波长参数 ====================
def add_wavelength_bands(image):
    spacecraft = ee.String(image.get('SPACECRAFT_ID'))
    wl_dict = ee.Dictionary({
        'LT05': ee.Dictionary({'red':660,'nir':835,'swir1':1650}),
        'LE07': ee.Dictionary({'red':660,'nir':835,'swir1':1650}),
        'LC08': ee.Dictionary({'red':655,'nir':865,'swir1':1609}),
        'LC09': ee.Dictionary({'red':655,'nir':865,'swir1':1609})
    })
    prefix = ee.Algorithms.If(
        spacecraft.compareTo('Landsat 5').eq(0), 'LT05',
        ee.Algorithms.If(spacecraft.compareTo('Landsat 7').eq(0), 'LE07',
        ee.Algorithms.If(spacecraft.compareTo('Landsat 8').eq(0), 'LC08',
        ee.Algorithms.If(spacecraft.compareTo('Landsat 9').eq(0), 'LC09','LT05'))))
    sat_wl = ee.Dictionary(wl_dict.get(prefix))
    return image.addBands(
        ee.Image.constant([
            sat_wl.getNumber('red'),
            sat_wl.getNumber('nir'),
            sat_wl.getNumber('swir1')
        ]).rename(['lambda_red','lambda_nir','lambda_swir1'])
    )

# ==================== 添加Hue和FAI指数 ====================
def add_hue_fai(image):
    # 180/π 常量
    rad2deg = ee.Image.constant(180.0).divide(math.pi)
    # Hue 计算
    hue = image.expression(
        'atan2(sqrt(3) * (G - B), (2 * R - G - B))',
        {'R': image.select('Rrs_B4'),
         'G': image.select('Rrs_B3'),
         'B': image.select('Rrs_B2')}
    ).multiply(rad2deg).rename('Hue')
    # FAI 计算
    fai = image.expression(
        'NIR - (R_red + (SWIR1 - R_red) * (lambda_nir - lambda_red) / (lambda_swir1 - lambda_red))',
        {'NIR': image.select('Rrs_B5'),
         'R_red': image.select('Rrs_B4'),
         'SWIR1': image.select('Rrs_B6'),
         'lambda_red': image.select('lambda_red'),
         'lambda_nir': image.select('lambda_nir'),
         'lambda_swir1': image.select('lambda_swir1')}
    ).rename('FAI')
    return image.addBands([hue, fai])

# ==================== 合并数据集 ====================
merged = ee.ImageCollection([])
for coll, bands in [(landsat5, landsat5_7_bands), 
                   (landsat7, landsat5_7_bands),
                   (landsat8, landsat8_9_bands),
                   (landsat9, landsat8_9_bands)]:
    processed = coll.map(scale_to_reflectance)\
                    .map(add_wavelength_bands)\
                    .map(add_hue_fai)
    merged = merged.merge(processed)




# 特征集合
table = ee.FeatureCollection("users/yaoyansum/GloDIC40000")

# QA掩膜
def apply_qa_mask(image):
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(1<<7)
    mask = mask.And(qa.bitwiseAnd(1<<1).eq(0))
    mask = mask.And(qa.bitwiseAnd(0b1100).eq(0))
    mask = mask.And(qa.bitwiseAnd(0b110000).eq(0))
    mask = mask.And(qa.bitwiseAnd(1<<5).eq(0))
    return image.updateMask(mask).copyProperties(image, ['system:time_start','SPACECRAFT_ID'])

# 转换日期
def convert_float_date(feature):
    date_float = ee.Number(feature.get('date'))
    date_str = date_float.toInt().format('%08d')
    return feature.set('parsed_date', ee.Date.parse('YYYYMMdd', date_str))

# 处理批次函数

def process_batch(batch_index, total_batches):
    feature_list = table.toList(table.size())
    total = feature_list.size()
    batch_size = math.ceil(total.getInfo()/total_batches)
    start = batch_index * batch_size
    end = start + batch_size
    current = feature_list.slice(start, end)
    feats = ee.FeatureCollection(current).map(convert_float_date)

    def match_images(feature):
        f_date = ee.Date(feature.get('parsed_date'))
        pt = ee.Geometry.Point([feature.getNumber('Lon'), feature.getNumber('Lat')])
        
        date_range = ee.DateRange(f_date.advance(-7,'day'), f_date.advance(7,'day'))
        
        imgs = merged.filterBounds(pt).filterDate(date_range).map(apply_qa_mask)

        def extract_data(image):
            stats = image.reduceRegion(ee.Reducer.mean(), pt, 30, maxPixels=1e9)

            return ee.Feature(None, {
                'ID': feature.get('Hylak_id'),
                'Field_date': feature.get('date'),
                'DIC': feature.get('DIC_mg_L_1'),
                'Lon': feature.get('Lon'),
                'Lat': feature.get('Lat'),
                'Dataset': feature.get('dataset'),
                'Hylak_id': feature.get('Hylak_id'),
                'Satellite': image.get('SPACECRAFT_ID'),
                'Acq_Date': ee.Date(image.get('system:time_start')).format('YYYY-MM-dd'),
                'Rrs_B2': stats.get('Rrs_B2'),
                'Rrs_B3': stats.get('Rrs_B3'),
                'Rrs_B4': stats.get('Rrs_B4'),
                'Rrs_B5': stats.get('Rrs_B5'),
                'Rrs_B6': stats.get('Rrs_B6'),
                'Hue': stats.get('Hue'),
                'FAI': stats.get('FAI')
            })
        return imgs.map(extract_data)

    results = feats.map(match_images).flatten()
    # 导出
    task = ee.batch.Export.table.toDrive(
        collection=results,
        description=f'WaterExtract_{batch_index+1}of{total_batches}',
        fileFormat='CSV',
        folder='WaterExtract_Results_WHindex',
        selectors=[
            'ID','Field_date','DIC','Lon','Lat','Dataset','Hylak_id',
            'Satellite','Acq_Date','Rrs_B2','Rrs_B3','Rrs_B4','Rrs_B5','Rrs_B6','Hue','FAI'
        ]
    )
    task.start()

# 执行
if __name__ == '__main__':
    total_batches = 10
    for i in range(total_batches):
        process_batch(i, total_batches)
        print(f'已提交批次 {i+1}/{total_batches}')


已提交批次 1/10
已提交批次 2/10
已提交批次 3/10
已提交批次 4/10
已提交批次 5/10
已提交批次 6/10
已提交批次 7/10
已提交批次 8/10
已提交批次 9/10
已提交批次 10/10


# 匹配最近日期

In [15]:
import ee
import math

# 初始化 Earth Engine
ee.Initialize()

# ==================== 数据预处理 ====================
def preprocess_landsat(collection, band_map):
    return collection.select(band_map['input'], band_map['output'])

landsat5_7_bands = {
    'input': ['SR_B1','SR_B2','SR_B3','SR_B4','SR_B5','SR_B7','QA_PIXEL'],
    'output': ['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7','QA_PIXEL']
}
landsat8_9_bands = {
    'input': ['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7','QA_PIXEL'],
    'output': ['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7','QA_PIXEL']
}

landsat5 = preprocess_landsat(ee.ImageCollection("LANDSAT/LT05/C02/T1_L2"), landsat5_7_bands)
landsat7 = preprocess_landsat(ee.ImageCollection("LANDSAT/LE07/C02/T1_L2"), landsat5_7_bands)
landsat8 = preprocess_landsat(ee.ImageCollection("LANDSAT/LC08/C02/T1_L2"), landsat8_9_bands)
landsat9 = preprocess_landsat(ee.ImageCollection("LANDSAT/LC09/C02/T1_L2"), landsat8_9_bands)

# ==================== 反射率转换 ====================
def scale_to_reflectance(image):
    optical = image.select(['SR_B2','SR_B3','SR_B4','SR_B5','SR_B6','SR_B7'])
    scaled = optical.multiply(0.0000275).add(-0.2)
    return image.addBands(scaled.rename(['Rrs_B2','Rrs_B3','Rrs_B4','Rrs_B5','Rrs_B6','Rrs_B7']))

# ==================== 添加波长参数 ====================
def add_wavelength_bands(image):
    spacecraft = ee.String(image.get('SPACECRAFT_ID'))
    wl_dict = ee.Dictionary({
        'LT05': ee.Dictionary({'red':660,'nir':835,'swir1':1650}),
        'LE07': ee.Dictionary({'red':660,'nir':835,'swir1':1650}),
        'LC08': ee.Dictionary({'red':655,'nir':865,'swir1':1609}),
        'LC09': ee.Dictionary({'red':655,'nir':865,'swir1':1609})
    })
    prefix = ee.Algorithms.If(
        spacecraft.compareTo('Landsat 5').eq(0), 'LT05',
        ee.Algorithms.If(spacecraft.compareTo('Landsat 7').eq(0), 'LE07',
        ee.Algorithms.If(spacecraft.compareTo('Landsat 8').eq(0), 'LC08',
        ee.Algorithms.If(spacecraft.compareTo('Landsat 9').eq(0), 'LC09','LT05'))))
    sat_wl = ee.Dictionary(wl_dict.get(prefix))
    return image.addBands(
        ee.Image.constant([
            sat_wl.getNumber('red'),
            sat_wl.getNumber('nir'),
            sat_wl.getNumber('swir1')
        ]).rename(['lambda_red','lambda_nir','lambda_swir1'])
    )

# ==================== 添加 Hue 和 FAI 指数 ====================
def add_hue_fai(image):
    # 弧度转度常量
    rad2deg = ee.Image.constant(180.0).divide(math.pi)
    # Hue
    hue = image.expression(
        'atan2(sqrt(3) * (G - B), (2 * R - G - B))',
        {'R': image.select('Rrs_B4'),
         'G': image.select('Rrs_B3'),
         'B': image.select('Rrs_B2')}
    ).multiply(rad2deg).rename('Hue')
    # FAI
    fai = image.expression(
        'NIR - (R_red + (SWIR1 - R_red) * '
        '(lambda_nir - lambda_red) / (lambda_swir1 - lambda_red))',
        {'NIR': image.select('Rrs_B5'),
         'R_red': image.select('Rrs_B4'),
         'SWIR1': image.select('Rrs_B6'),
         'lambda_red': image.select('lambda_red'),
         'lambda_nir': image.select('lambda_nir'),
         'lambda_swir1': image.select('lambda_swir1')}
    ).rename('FAI')
    return image.addBands([hue, fai])

# 合并并预处理所有影像集合
merged = ee.ImageCollection([])
for coll in [landsat5, landsat7, landsat8, landsat9]:
    processed = coll.map(scale_to_reflectance)\
                    .map(add_wavelength_bands)\
                    .map(add_hue_fai)
    merged = merged.merge(processed)

# ==================== QA 掩膜 ====================
def apply_qa_mask(image):
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(1<<7)
    mask = mask.And(qa.bitwiseAnd(1<<1).eq(0))
    mask = mask.And(qa.bitwiseAnd(0b1100).eq(0))
    mask = mask.And(qa.bitwiseAnd(0b110000).eq(0))
    mask = mask.And(qa.bitwiseAnd(1<<5).eq(0))
    return image.updateMask(mask).copyProperties(image, ['system:time_start','SPACECRAFT_ID'])

# ==================== 日期转换 ====================
def convert_float_date(feature):
    date_float = ee.Number(feature.get('date'))
    date_str = date_float.toInt().format('%08d')
    return feature.set('parsed_date', ee.Date.parse('YYYYMMdd', date_str))

# ==================== 匹配并提取最新影像数据 ====================
def match_images(feature):
    # 解析日期、点位
    f_date = ee.Date(feature.get('parsed_date'))
    pt     = ee.Geometry.Point([feature.getNumber('Lon'), feature.getNumber('Lat')])
    date_range = ee.DateRange(f_date.advance(-7, 'day'),
                              f_date.advance( 7, 'day'))

    # 空间+时间+QA 过滤
    imgs = (merged
            .filterBounds(pt)
            .filterDate(date_range)
            .map(apply_qa_mask))
    
    # 取最新影像
    latest = ee.Image(imgs.sort('system:time_start', False).first())

    # 如果没有影像，返回空 Feature
    return ee.Algorithms.If(
        latest,
        # 否则做一次 reduceRegion 并构造 Feature
        ee.Feature(None) \
          .set({
              'ID':         feature.get('Hylak_id'),
              'Field_date': feature.get('date'),
              'DIC':        feature.get('DIC_mg_L_1'),
              'Lon':        feature.get('Lon'),
              'Lat':        feature.get('Lat'),
              'Dataset':    feature.get('dataset'),
              'Hylak_id':   feature.get('Hylak_id'),
              'Satellite':  latest.get('SPACECRAFT_ID'),
              'Acq_Date':   ee.Date(latest.get('system:time_start')).format('YYYY-MM-dd')
          }) \
          .set(latest.reduceRegion(
              reducer=ee.Reducer.mean(),
              geometry=pt,
              scale=30,
              maxPixels=1e9
          )),
        # 无影像时：空 Feature（可后续过滤掉）
        ee.Feature(None)
    )

# ==================== 批次处理与导出 ====================
def process_batch(batch_index, total_batches):
    table = ee.FeatureCollection("users/yaoyansum/GloDIC40000")
    total = table.size().getInfo()
    batch_size = math.ceil(total / total_batches)
    start = batch_index * batch_size
    feats = ee.FeatureCollection(table.toList(total).slice(start, start + batch_size))\
                .map(convert_float_date)

    results = feats.map(match_images)

    task = ee.batch.Export.table.toDrive(
        collection=results,
        description=f'WaterExtract_{batch_index+1}of{total_batches}',
        folder='WaterExtract_Results_WHindex_NearTM',
        fileFormat='CSV',
        selectors=[
            'ID','Field_date','DIC','Lon','Lat','Dataset','Hylak_id',
            'Satellite','Acq_Date','Rrs_B2','Rrs_B3','Rrs_B4',
            'Rrs_B5','Rrs_B6','Hue','FAI'
        ]
    )
    task.start()
    print(f'已提交批次 {batch_index+1}/{total_batches}')

# 主流程
if __name__ == '__main__':
    total_batches = 10
    for i in range(total_batches):
        process_batch(i, total_batches)


已提交批次 1/10
已提交批次 2/10
已提交批次 3/10
已提交批次 4/10
已提交批次 5/10
已提交批次 6/10
已提交批次 7/10
已提交批次 8/10
已提交批次 9/10
已提交批次 10/10


# 一般代码

In [ ]:
import numpy as np
import pandas as pd
import ee
import geemap
import os

# ee.Authenticate()
ee.Initialize()
# 加载 Landsat 9 遥感影像
landsat =  ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")                        ####需要修改
# 加载 FeatureCollection
table = ee.FeatureCollection("projects/ee-yannuoxiao01/assets/Stations_GEE_withinTl8")             ####需要修改
# 定义一个函数来匹配 FeatureCollection 和 Landsat 影像
def match_images(feature):
    feature_date = ee.Date(feature.getString('Field date'),'Asia/Shanghai')
    feature_lati = ee.Number(feature.get("Lati_Mod"))
    feature_longi = ee.Number(feature.get("Long_Mod"))
    feature_no = ee.Number(feature.get("No"))
    point = ee.Geometry.Point(feature.getNumber('Long_Mod'),feature.getNumber('Lati_Mod'))
    
    # 定义时间窗口
    date_range = ee.DateRange(feature_date.advance(-30, "day"), feature_date.advance(30, "day"))
    
    # 过滤符合条件的 Landsat 影像
    matched_images = landsat.filterBounds(point).filterDate(date_range)
    bands_select=['SR_B2','SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7']
    def extract_info(image):
        image_date = ee.Date(image.get("system:time_start"))
        image_info0 = image.select(bands_select).multiply(0.0000275).add(-0.2)
        image_info = image_info0.addBands(image.select('QA_PIXEL'))
        return ee.Feature(None, {
            'imagetime(UTC)': image_date.format(),
            'No': feature_no,
            "Long_Mod":feature_longi,
            "Lati_Mod":feature_lati,
            'SR_B2': image_info.reduceRegion(reducer=ee.Reducer.mean(),geometry=point,scale=30).get('SR_B2'),
            'SR_B3': image_info.reduceRegion(reducer=ee.Reducer.mean(),geometry=point,scale=30).get('SR_B3'),
            'SR_B4': image_info.reduceRegion(reducer=ee.Reducer.mean(),geometry=point,scale=30).get('SR_B4'),
            'SR_B5': image_info.reduceRegion(reducer=ee.Reducer.mean(),geometry=point,scale=30).get('SR_B5'),
            'SR_B6': image_info.reduceRegion(reducer=ee.Reducer.mean(),geometry=point,scale=30).get('SR_B6'),
            'SR_B7': image_info.reduceRegion(reducer=ee.Reducer.mean(),geometry=point,scale=30).get('SR_B7'),
            'QA_PIXEL':image_info.reduceRegion(reducer=ee.Reducer.mean(),geometry=point,scale=30).get('QA_PIXEL')
        })

    # 提取影像信息
    image_info = matched_images.map(extract_info)

    return image_info
    # 映射 FeatureCollection，获取所有匹配的信息
image_data = table.map(match_images).flatten()
# 创建一个 GEE 表格
ee_table = ee.FeatureCollection(image_data)

# 将 GEE 表格导出为 CSV 文件
task = ee.batch.Export.table.toDrive(
    collection=ee_table,
    description="Stations_GEE_l8_wt30",
    fileFormat="CSV",
    selectors=['No','imagetime(UTC)', 'Long_Mod', 'Lati_Mod', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5','SR_B6','SR_B7','QA_PIXEL']
)                                                                                                                              ####需要修改
task.start()